In [1]:
# This is an automation of code provided in book Tom Bäckström, Okko Räsänen, Abraham Zewoudie, Pablo Pérez Zarazaga, Liisa Koivusalo,
# Sneha Das, Esteban Gómez Mellado, Mariem Bouafif Mansali, Daniel Ramos, Sudarsana Kadiri and Paavo Alku “Introduction to Speech
# Processing”, 2nd Edition, 2022. URL: https://speechprocessingbook.aalto.fi, DOI: 10.5281/zenodo.6821775.

# In order to use this code you need to have .csv file containing the time intervals for intended words or sentences for your analysis

# Get the path to the desktop directory on macOS
import os
user_directory = os.path.join(os.path.expanduser('~'), '')
os.chdir(user_directory)

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import librosa.display
import numpy as np
from scipy.linalg import solve_toeplitz, toeplitz
import scipy


In [7]:

# Load the CSV file into a DataFrame
input_csv_file = './Desktop/Folders/.csv'  # Replace with your CSV file path
df = pd.read_csv(input_csv_file)

# Load the raw audio file
audio_file = './Desktop/Folders/.wav'  # Replace with your audio file path
audio, sample_rate = librosa.load(audio_file, sr=None)

subject = 'Erfan'
output_directory = './Desktop/Folders/Frequency_plots/Formant/' + subject + '/'  # Specify the directory where you want to save the plots

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)
counter = 0

for index, row in df.iterrows():
    counter = counter+1
    word_name = row['word']
    start_time = row['start']
    end_time = row['end']
    
    # Convert start and end times to sample indices
    start_sample = int(start_time * sample_rate)
    end_sample = int(end_time * sample_rate)

    # Extract the audio segment within the specified time interval
    word_audio = audio[start_sample:end_sample]

    window_length = len(word_audio)

    windowing_function = np.sin(np.pi*np.arange(0.5,window_length,1)/window_length)**2
    frequency_vector = np.linspace(0,sample_rate/2000,int(window_length/2+1))
    signal_spectrum = np.abs(scipy.fft.rfft(word_audio*windowing_function)) 


    autocorrelation = scipy.fft.irfft(np.abs(scipy.fft.rfft(word_audio*windowing_function))**2) 
                                                                                                
    lpc_order = int(sample_rate/1000 + 2)
    u = np.zeros([lpc_order+1,1])
    u[0]=1
    lpc_model = solve_toeplitz(autocorrelation[0:lpc_order+1],u)
    lpc_model /= lpc_model[0]
    envelope_spectrum = np.abs(scipy.fft.rfft(lpc_model,window_length,axis=0))**-1
    signal_spectrum = np.abs(scipy.fft.rfft(word_audio*windowing_function))
    envelope_spectrum *= np.max(signal_spectrum)/np.max(envelope_spectrum)

    # Find peaks of envelope
    diff_envelope = np.diff(envelope_spectrum,axis=0)
    sign_diff_envelope = np.sign(diff_envelope)
    diff_sign_diff_envelope = np.diff(sign_diff_envelope,axis=0)
    peak_indices = np.argwhere(diff_sign_diff_envelope[:,0] < 0)+1
    peak_indices = peak_indices[0:5]

    plt.figure(figsize=[12,4])

    plt.subplot(121)
    plt.plot(frequency_vector,20*np.log10(signal_spectrum),label='Spectrum')
    plt.plot(frequency_vector,20*np.log10(envelope_spectrum),linewidth=3,label='Envelope')
    plt.plot(frequency_vector[peak_indices],20*np.log10(envelope_spectrum[peak_indices,0]),marker='^',linestyle='',label='Peaks')
    for k in range(len(peak_indices)): 
        x = frequency_vector[peak_indices[k]]
        y = 20*np.log10(envelope_spectrum[peak_indices[k],0]) + 5
        plt.text(x,y,'F' + str(k+1))    
        plt.legend()
        plt.title(str(word_name)+' - '+ subject +'  - \nLog-magnitude spectrum and its envelope')
        plt.xlabel('Frequency (kHz)')
        plt.ylabel('Magnitude $20\log_{10}|X_k|$')
        ax = plt.axis()
        ax = [ax[0], ax[1], ax[2], ax[3]+5]
        plt.axis(ax)

    plt.subplot(122)
    plt.plot(frequency_vector,20*np.log10(signal_spectrum),label='Spectrum')
    plt.plot(frequency_vector,20*np.log10(envelope_spectrum),linewidth=3,label='Envelope')
    plt.plot(frequency_vector[peak_indices],20*np.log10(envelope_spectrum[peak_indices,0]),marker='^',linestyle='',label='Peaks')
    for k in range(len(peak_indices)): 
        x = frequency_vector[peak_indices[k]]
        y = 20*np.log10(envelope_spectrum[peak_indices[k],0]) + 5
        if x < 8:  plt.text(x,y,'F' + str(k+1))    
    plt.legend()
    plt.title(str(word_name)+' - '+ subject +'  - \nLog-magnitude spectrum zoomed to [0,8kHz]')
    plt.xlabel('Frequency (kHz)')
    plt.ylabel('Magnitude $20\log_{10}|X_k|$ (dB)')
    ax = [0, 8, ax[2], ax[3]+5]
    plt.axis(ax)

    plt.tight_layout()

    figure_filename = os.path.join(output_directory, str(counter)+'_'+f'{word_name}.jpg')
    plt.savefig(figure_filename, dpi=300)
    plt.close() 
